In [72]:

import pandas as pd
import openpyxl
from datetime import datetime
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [73]:
#Reading both the sheets of data
df1 = pd.read_excel("Copy of customer_transactions_sample.xlsx",sheet_name='Year 2009-2010')
df2 = pd.read_excel("Copy of customer_transactions_sample.xlsx",sheet_name='Year 2010-2011')

In [74]:
#Combining the data in both the sheets
df = pd.concat([df1, df2], ignore_index=True)

# Converting the date in invoice to datetime format
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

#Finding the total price of an item purchased
df['TotalPrice'] = df['Quantity'] * df['Price']

#Converting the Nominal variabes to strings
df['Invoice'] = df['Invoice'].astype(str)
df['Customer ID'] = df['Customer ID'].astype(str)

#Removing the cancelled Transactions
df_cancelled = df[df['Invoice'].str.startswith('C')]
df = df[~df['Invoice'].str.startswith('C')]

In [75]:
#Creating the RFM table aggregating the customer ID
rfm_table = df.groupby('Customer ID').agg({'InvoiceDate': lambda x: (df['InvoiceDate'].max() - x.max()).days,
                                      'Invoice': lambda x: x.count(),
                                      'TotalPrice': lambda x: x.mean()}).reset_index()
rfm_table.columns = ['Customer ID', 'Recency', 'Frequency', 'Monetary']

# Removing the unknown customer IDs
rfm_table = rfm_table.drop(rfm_table.index[-1])

In [83]:
#Scaling the RFM table without customer ID as it is nominal
scaler = StandardScaler()
rfm_table_scaled = scaler.fit_transform(rfm_table.drop(columns=['Customer ID']))

#Making cluters using KMeans
kmeans = KMeans(n_clusters=3, random_state=42)
rfm_table_clusters = kmeans.fit_predict(rfm_table_scaled)
rfm_table['Cluster'] = rfm_table_clusters

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [84]:
import plotly.express as px

#Plotting the clusters with respect to customer ID
fig = px.scatter(rfm_table, x='Customer ID', y='Frequency', color='Cluster', opacity=1)
fig.show()
fig = px.scatter(rfm_table, x='Customer ID', y='Recency', color='Cluster', opacity=1)
fig.show()
fig = px.scatter(rfm_table, x='Customer ID', y='Monetary', color='Cluster', opacity=1)
fig.show()

In [85]:
#3D plot for RFM data
fig = px.scatter_3d(rfm_table, x='Recency', y='Frequency', z='Monetary', color='Cluster', opacity=0.7,
                    title='3D Scatter Plot of Recency, Frequency, and Monetary with Clusters')
fig.show()

In [86]:
#The mean for RFM based on cluster
customer_data = df.merge(rfm_table, on='Customer ID')
print(customer_data.groupby('Cluster').agg({'Recency': 'mean', 'Frequency': 'mean', 'Monetary': 'mean'}))

            Recency    Frequency      Monetary
Cluster                                       
0         39.797835  1161.347123     22.125797
1        431.094964   129.894248     19.238741
2          0.000000     3.000000  56157.500000


### Findings

From the above we can observe that
- Cluster 0 has the highest frequency customers.
- Cluster 1 has high recency of customers.
- Cluster 2 has the highest monetary customers.

In [87]:
#Findling the loyal customers
rfm_table['Disloyal'] = rfm_table['Customer ID'].isin(df_cancelled['Customer ID'])

In [92]:
#Plotting the statistics of different tyoe of customers

recent_customers_count = (rfm_table['Cluster'] == 1).sum()
frequent_customers_count = (rfm_table['Cluster'] == 0).sum()
big_spenders_count = (rfm_table['Cluster'] == 2).sum()
loyal_customers_count = (rfm_table['Disloyal'] == False).sum()
categories = ['Recent Customers', 'Frequent Customers', 'Big Spenders', 'Loyal Customers']
counts = [recent_customers_count, frequent_customers_count, big_spenders_count, loyal_customers_count]

fig = go.Figure(data=[go.Bar(x=categories, y=counts)])
fig.update_layout(title='Customer Categories',
                  xaxis_title='Categories',
                  yaxis_title='Count')
fig.show()